# polARosol: for a brighter future?
## **POL**ar **A**tmospheric **R**ivers and how they are influenced by Ae**ROSOL**s and how that might affect cloud **BRIGHT**ness but mostly other parameters now and in the **FUTURE**


Lea Haberstock 

lea.haberstock@gmail.com

eScience course 2022, Group 5, December 2nd 2022

Group Assistant: Rémy Lapere

## Abstract
1/2 page max

## Introduction

1 page max

- What is an atmospheric River
- why do they matter
- arctic vs antarctic
- research questions
- hypotheses
- scope of work

The polar regions are highly sensitive to changes in temperature and energy flux due to different positive feedback mechanisms. As a result of this sensitivity, the polar regions warm faster than the global mean temperature, an effect also known as Polar amplification (e.g., Manabe and Wetherald, 1975). The surface albedo feedback describes one of the main effects of a positive enhancement of temperature increase and was first described by Arrhenius in 1896. With rising temperatures, ice and snow will melt and decrease the albedo in high latitudes, resulting in higher absorption of radiation and temperature. 

Increasing downward longwave radiation was found to play an important role regarding sea ice retreat. In die Arctic horizontal moisture transported and warm air advection from lower latitudes are the main cause for an increase in the downward longwave radiation (D.-S. R. Park et al. 2015). Woods et al. (2013) found that small and thin but intense moisture injections are the main driver of variance in arctic mean surface air temperature and downward longwave radiation. In Antarctica, moist and warm air intrusions are considered to play a critical role in surface melting on major ice shelves due to an increase in integrated water vapor and cloud liquid and ice water content. 

These intense moisture injections are also known as Atmospheric rivers (AR) which are vapor rich meridional air masses originating from the tropics or subtropics. An atmospheric river is defined as 'A long, narrow, and transient corridor of strong horizontal water vapor transport that is typically associated with a low-level jet stream ahead of the cold front of an extratropical cyclone' (Glossary of Meteorology, 2020). ARs are responsible for over 90 % of meridional vapor transport in the mid latitudes (Nash et al., 2018). In polar regions ARs perturb the usually thermodynamically stable systems and enhance temperature, humidity, precipitation, cloud cover and can change the aerosol number concentration and size distribution (Dada et al., 2022; Wille et al., 2021). 

In the Arctic, the frequency of AR occurence increased in the past decades (Dada et al., 2022). With a changing climate and polar amplification, the influcence of ARs on the polar systems might change from the effects that are seen now. To investigate the role of polar ARs in the future



Important mechanisms:

- downward longwave radiation --> attributed to influx of moisture: radiatively opaque (D.-S. R. Park et al. 2015)
- small, intense thin moisture injections significant influence on variance in arctic mean suface air temperature and lw radiation (Woods et al. 2013)


 
- enhanced radiative fluxes due to vapor in clouds --> sea ice loss/disintegration
- ice mass losses over ice sheet in greenland
- sea ice melt preconditioning (arctic?)
- large scale surface melt in antarctica
- can also lead to heavy snowfall (antarctica 74-80 % of accumulated snowfall (but extremely many AR in 2009-2011))
- can bring aerosols /polluted areas

Research question: 
*1. 
*2. How will occurence and intensity of polar AR and their effect on clouds, aerosol transport, and precipitation change in the future?*

Hypothesis:
1. AR become more frequent and stronger with stronger climate change 
2. 



## Data and Methods

- Analysis method
    - comparing frequency distributions of temperature, humidity, precipitation, aod and cloud cover inside and outside of AR
    - comparing frequency distributions of temperature anomaly, humidity, precipitation, aod and cloud cover between different scenarios 


 ...
- NorESM historical data 2000-2014 and scenarios ssp245, 370 and 585 in 2085-2100
- AR detection algorithm
- analysis of parameters

Packages

In [ ]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import s3fs
import intake
import cftime
import numpy as np
from netCDF4 import Dataset
import scipy.stats as st
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.path as mpath
# my functions
from all_functions import *
xr.set_options(display_style='html')

Datasets

**NorESM-LM2** The Norwegian Earth System Model (NorESM) is part of the sixth Coupled Model Intercomparison Project (CMIP6). The scenarios ssp245, ssp370, and ssp585, (years 2085-2100) and the 'historical' (years 2000-2014) experiments with the run 'r1i1p1f1' were selected. The investigated variables are precipitation (pr), cloud cover (clt), specific humidity (hus), poleward wind component (va), and surface temperature (tas).

**Atmospheric River Detection Algorithm** The basis for the atmospheric river detection algorithm (ARDA) used in this work was developed by Willie et al. () and adapted for this project during data analysis. Atmospheric rivers are detected via a threshold for integrated water vapor (IVT) multiplied with the meridional wind component. The threshold is defined by calculating the 94th quantile of IVT in every grid cell of the historical data. An atmospheric river is then only detected if a cluster of connected grid cells over 20 ° lat exceed the calculated threshold. The code for the ARDA in this project was written by Rémy Lapere. 

In [ ]:
# import data (historical, ssp245, ssp370, ssp585 respectively) for both poles (s = Antarctic, n = Arctic)
from all_functions import nh, sh, n245, s245, n370, s370, n585, s585 

# import detected Atmospheric Rivers (ivt = True)
arh = xr.open_dataset('historical_combined_q94.nc')
ar245 = xr.open_dataset('ssp245_combined_q94.nc')
ar370 = xr.open_dataset('ssp370_combined_q94.nc')
ar585 = xr.open_dataset('ssp585_combined_q94.nc')

## mask data for being inside (masked) or outside (neg) of a atmospheric river
# arctic
masked_n245 = n245.where(ar245['ivt']==True)
masked_n370 = n370.where(ar370['ivt']==True)
masked_n585 = n585.where(ar585['ivt']==True)
masked_nh = nh.where(arh['ivt']==True)
neg_n245 = n245.where(ar245['ivt']==False)
neg_n370 = n370.where(ar370['ivt']==False)
neg_n585 = n585.where(ar585['ivt']==False)
neg_nh = nh.where(arh['ivt']==False)

# antarctic
masked_s245 = s245.where(ar245['ivt']==True)
masked_s370 = s370.where(ar370['ivt']==True)
masked_s585 = s585.where(ar585['ivt']==True)
masked_sh = sh.where(arh['ivt']==True)
neg_s245 = s245.where(ar245['ivt']==False)
neg_s370 = s370.where(ar370['ivt']==False)
neg_s585 = s585.where(ar585['ivt']==False)
neg_sh = sh.where(arh['ivt']==False)

## treating data for frequency analysis; flatten arrays and remove vectors
#int_hum = integrated humidity, aod = aerosol optical depth, precip = preicipitation, temp = surface temperature anomaly, cloud = cloud cover
from all_functions import int_hum, aod, preicp, temp, cloud


## Results
### Comparison inside and outside AR

### Trends in Future Scanrios

## Discussion and Outlook 
1 page

## Conclusion
1/2 page


## References
American Meteorological Society. 2020, Glossary of Meteorology, url: https://glossary.ametsoc.org/wiki/Atmospheric_river, last accessed at 16.11.22

Woods C and Caballero R., 2016, The Role of Moist Intrusions in Winter Arctic Warming and Sea Ice Decline

Manabe and Wetherald, 1975

Woods et al., 2013, Large-scale circulation associated with moisture intrusions into the Arctic during winter, 

(D.-S. R. Park et al. 2015)
Dada et al., 2022, A central arctic extreme aerosol event triggered by a warm air-mass intrusion

## Acknowledgments (helpers, collaborators, model and data owners/providers)
## Supplementary material (extra figures, functions, pre-processing routines)